In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
questions = pd.read_csv('./question_train_word.csv')
questions['topics'] = questions.topics.apply(lambda s: s.split(','))
questions['title'] = questions.title.astype('U').apply(lambda s: s.split(','))

CPU times: user 28 s, sys: 7.99 s, total: 36 s
Wall time: 40 s


In [3]:
from collections import defaultdict

In [4]:
questions.iloc[0]

Unnamed: 0                                                      0
Unnamed: 0.1                                                    0
id                                            6555699376639805223
title           [w305, w13549, w22752, w11, w7225, w2565, w110...
description     w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1...
topics                 [7739004195693774975, 3738968195649774859]
Name: 0, dtype: object

In [9]:
from tqdm import tqdm

In [12]:
topic_words = defaultdict(lambda: [])
for i, q in tqdm(questions.iterrows()):
    for t in q[5]:
        topic_words[t].extend(q[3])

2999967it [06:18, 7928.38it/s]


In [15]:
import sys
print(sys.getsizeof(topic_words))
len(topic_words.keys())

73832


1999

In [16]:
topics = topic_words.keys()
Word_Vectors = []
for t in tqdm(topics):
    Word_Vectors.append(' '.join(topic_words[t]))

100%|██████████| 1999/1999 [00:15<00:00, 133.21it/s]


In [18]:
len(Word_Vectors[0])

173439

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer()

In [21]:
%%time
Word_Count = count_vec.fit_transform(Word_Vectors)

CPU times: user 1min 18s, sys: 3.42 s, total: 1min 22s
Wall time: 1min 25s


In [22]:
Word_Count.shape

(1999, 324960)

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()

In [24]:
%%time
Word_tfidf = tfidf.fit_transform(Word_Count)

CPU times: user 1.91 s, sys: 144 ms, total: 2.06 s
Wall time: 2.09 s


In [25]:
from scipy.spatial import cKDTree

In [35]:
# %%time
print(Word_tfidf.shape)
print(type(Word_tfidf))
print(type(Word_tfidf[0]))

(1999, 324960)
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>


In [62]:
from sklearn.preprocessing import normalize

In [63]:
X_kd = []
for i in range(1999):
    X_kd.append(normalize(Word_tfidf[i]))

In [70]:
from sklearn.neighbors import LSHForest
lshf = LSHForest(n_estimators=30)

In [71]:
%%time
lshf.fit(Word_tfidf)

CPU times: user 39.2 s, sys: 2.01 s, total: 41.2 s
Wall time: 43.6 s


LSHForest(min_hash_match=4, n_candidates=50, n_estimators=30, n_neighbors=5,
     radius=1.0, radius_cutoff_ratio=0.9, random_state=None)

In [72]:
lshf.kneighbors(normalize(X_kd[99]))

(array([[ 0.        ,  0.03858089,  0.11217432,  0.11352881,  0.1212609 ]]),
 array([[  99, 1114,   57,  471,  339]]))

In [47]:
%%time
titles = questions.title.apply(lambda x: ' '.join(x))
Title_counts = count_vec.transform(titles)

CPU times: user 52.1 s, sys: 6.22 s, total: 58.3 s
Wall time: 1min 2s


In [48]:
Title_tfidf = tfidf.transform(Title_counts)

In [49]:
Title_tfidf.shape

(2999967, 324960)

In [74]:
lshf.kneighbors(Title_tfidf[1])
# (array([[ 0.7458829 ,  0.74771425,  0.75860983,  0.76099354,  0.76237584]]),
#  array([[951,   6,  11, 945,   9]]))
# (array([[ 0.75400866,  0.75927334,  0.76099004,  0.76099354,  0.76398319]]),
#  array([[ 207, 1269,  917,  945, 1358]]))

(array([[ 0.7458829 ,  0.74771425,  0.75860983,  0.75927334,  0.76009832]]),
 array([[ 951,    6,   11, 1269,  382]]))

In [68]:
print(questions.iloc[1].topics)
topics = list(topics)
print(topics[207], topics[1269], topics[917], topics[945], topics[9])

['-3149765934180654494']
-4749298169084869151 -9193714286190364511 -1201071336132923959 3250728804941389085 -1689319711084901730


In [59]:
from sklearn.preprocessing import normalize

In [60]:
x = normalize(X_kd[0])

In [61]:
x

<1x324960 sparse matrix of type '<class 'numpy.float64'>'
	with 4031 stored elements in Compressed Sparse Row format>